In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import holidays

In [2]:
df = pd.read_csv(
    "../dataa/LD2011_2014.txt",
    sep=";",                # AYRAÇ noktalı virgül
    quotechar='"',          # değerler " " içinde
    decimal=",",            # ondalık ayracı virgül
)

df = df.rename(columns={"Unnamed: 0": "Date"})
df.index = pd.to_datetime(df.iloc[:, 0])
df = df.drop(df.columns[0], axis=1)
df = df.sort_index()
df = df.asfreq("15min")
df.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#kW to kWh
df = df / 4

total_15min= df.sum(axis=1)

hourly_total = total_15min.resample("h").sum()
daily_total = total_15min.resample("D").sum()


In [4]:
total_15min.head()

Date
2011-01-01 00:15:00    17128.278835
2011-01-01 00:30:00    17295.076090
2011-01-01 00:45:00    17341.212643
2011-01-01 01:00:00    17087.620165
2011-01-01 01:15:00    16541.718576
Freq: 15min, dtype: float64

In [7]:
hourly_total.loc['2014-11-10']

Date
2014-11-10 00:00:00    136135.671043
2014-11-10 01:00:00    116629.662277
2014-11-10 02:00:00    115011.662495
2014-11-10 03:00:00    114674.560038
2014-11-10 04:00:00    117599.854405
2014-11-10 05:00:00    120152.218830
2014-11-10 06:00:00    134789.863772
2014-11-10 07:00:00    148699.421827
2014-11-10 08:00:00    159806.835153
2014-11-10 09:00:00    199292.052207
2014-11-10 10:00:00    226524.775793
2014-11-10 11:00:00    233306.742919
2014-11-10 12:00:00    232629.831605
2014-11-10 13:00:00    235879.395399
2014-11-10 14:00:00    234527.330433
2014-11-10 15:00:00    236960.641470
2014-11-10 16:00:00    240713.679584
2014-11-10 17:00:00    255728.566849
2014-11-10 18:00:00    262407.164732
2014-11-10 19:00:00    262101.279734
2014-11-10 20:00:00    255594.680438
2014-11-10 21:00:00    247296.339633
2014-11-10 22:00:00    221251.346877
2014-11-10 23:00:00    184101.644023
Freq: h, dtype: float64

In [6]:
daily_total.head()

Date
2011-01-01    1.713244e+06
2011-01-02    2.785336e+06
2011-01-03    2.812268e+06
2011-01-04    2.866966e+06
2011-01-05    2.880437e+06
Freq: D, dtype: float64

In [7]:
df_feat = hourly_total.to_frame(name="consumption")
df_feat.head()

,consumption
Date,
2011-01-01 00:00:00,51764.567568
2011-01-01 01:00:00,66344.627687
2011-01-01 02:00:00,65981.054883
2011-01-01 03:00:00,66576.533566
2011-01-01 04:00:00,64963.552675


In [8]:
# Time Features
df_feat["hour"] = df_feat.index.hour # Tüketimin hangi saatlerde yükseldiğini öğrenir
df_feat["dayofweek"] = df_feat.index.dayofweek # Hafta içi çalışma yükü
df_feat["month"] = df_feat.index.month #Klima / ısıtma sezonlarını ayırt eder
df_feat["is_weekend"] = df_feat["dayofweek"].isin([5,6]).astype(int) #Hafta sonu mu?


In [9]:
# Cyclieanc Encode
#Saatin döngüsel doğası
df_feat["hour_sin"] = np.sin(2*np.pi*df_feat["hour"]/24)
df_feat["hour_cos"] = np.cos(2*np.pi*df_feat["hour"]/24)
#Yıllık döngü
df_feat["month_sin"] = np.sin(2*np.pi*df_feat["month"]/12)
df_feat["month_cos"] = np.cos(2*np.pi*df_feat["month"]/12)


In [10]:
# Lag Features

df_feat["lag_1h"] = df_feat["consumption"].shift(1)#Bir saat önce ne oldu?
df_feat["lag_24h"] = df_feat["consumption"].shift(24)#Bir gün önce ne oldu?
df_feat["lag_168h"] = df_feat["consumption"].shift(168)#Bir hafta önce ne oldu?


In [11]:
# Rolling Features

df_feat["rolling_mean_24h"] = df_feat["consumption"].rolling(24).mean()#Kısa vadeli davranış(1gün)
df_feat["rolling_mean_7d"] = df_feat["consumption"].rolling(168).mean()#Orta vadeli davranış(1hafta)


In [16]:
# Bu kütüphane otamtaik olarak resmi tatilleri yıllara göre görür
pt_holidays = holidays.country_holidays('PT')

df_feat["is_holiday"] = df_feat.index.map(
    lambda x: 1 if x in pt_holidays else 0
)


In [17]:
df_feat.head()

,consumption,hour,dayofweek,month,is_weekend,hour_sin,hour_cos,month_sin,month_cos,lag_1h,lag_24h,lag_168h,rolling_mean_24h,rolling_mean_7d,is_holiday
Date,,,,,,,,,,,,,,,
2011-01-01 00:00:00,51764.567568,0,5,1,1,0.000000,1.000000,0.5,0.866025,NaN,NaN,NaN,NaN,NaN,1
2011-01-01 01:00:00,66344.627687,1,5,1,1,0.258819,0.965926,0.5,0.866025,51764.567568,NaN,NaN,NaN,NaN,1
2011-01-01 02:00:00,65981.054883,2,5,1,1,0.500000,0.866025,0.5,0.866025,66344.627687,NaN,NaN,NaN,NaN,1
2011-01-01 03:00:00,66576.533566,3,5,1,1,0.707107,0.707107,0.5,0.866025,65981.054883,NaN,NaN,NaN,NaN,1
2011-01-01 04:00:00,64963.552675,4,5,1,1,0.866025,0.500000,0.5,0.866025,66576.533566,NaN,NaN,NaN,NaN,1


In [27]:
# Örneğin Portekizde 8 Aralık Günahsız Doğuş Bayramıdır (Milli Bayram)
df_feat.loc["2012-12-08"].head()

,consumption,hour,dayofweek,month,is_weekend,hour_sin,hour_cos,month_sin,month_cos,lag_1h,lag_24h,lag_168h,rolling_mean_24h,rolling_mean_7d,is_holiday
Date,,,,,,,,,,,,,,,
2012-12-08 00:00:00,145033.537448,0,5,12,1,0.000000,1.000000,-2.449294e-16,1.0,178548.257179,132622.629106,140603.603160,187855.228043,181645.067335,1
2012-12-08 01:00:00,122888.061140,1,5,12,1,0.258819,0.965926,-2.449294e-16,1.0,145033.537448,110593.084828,118288.339005,188367.518723,181672.446634,1
2012-12-08 02:00:00,114701.837380,2,5,12,1,0.500000,0.866025,-2.449294e-16,1.0,122888.061140,107772.334640,110228.376501,188656.248004,181699.074377,1
2012-12-08 03:00:00,104385.330208,3,5,12,1,0.707107,0.707107,-2.449294e-16,1.0,114701.837380,103993.531862,101567.236315,188672.572935,181715.848746,1
2012-12-08 04:00:00,100056.161680,4,5,12,1,0.866025,0.500000,-2.449294e-16,1.0,104385.330208,105882.972006,99217.902311,188429.789171,181720.838385,1


In [28]:
import requests
#Lisbona göre sıcaklık verisini Open-Meteo API den çekiyoruz
url = (
    "https://archive-api.open-meteo.com/v1/archive?"
    "latitude=38.72&longitude=-9.13"
    "&start_date=2011-01-01&end_date=2014-12-31"
    "&hourly=temperature_2m,relativehumidity_2m"
    "&timezone=Europe/Lisbon"
)

response = requests.get(url)
data = response.json()

weather = pd.DataFrame({
    "Date": pd.to_datetime(data["hourly"]["time"]),
    "temperature": data["hourly"]["temperature_2m"],
    "humidity": data["hourly"]["relativehumidity_2m"]
})

weather = weather.set_index("Date")
weather.head()

,temperature,humidity
Date,,
2011-01-01 00:00:00,12.4,93
2011-01-01 01:00:00,12.3,94
2011-01-01 02:00:00,12.2,94
2011-01-01 03:00:00,12.2,94
2011-01-01 04:00:00,12.2,94


In [30]:
weather.loc['2012-08-10'].head(15)

,temperature,humidity
Date,,
2012-08-10 00:00:00,22.2,70
2012-08-10 01:00:00,21.4,75
2012-08-10 02:00:00,20.8,80
2012-08-10 03:00:00,20.4,82
2012-08-10 04:00:00,20.1,83
2012-08-10 05:00:00,20.0,83
2012-08-10 06:00:00,20.0,83
2012-08-10 07:00:00,20.3,83
2012-08-10 08:00:00,21.9,72


In [32]:
df_feat = df_feat.join(weather, how="left")


In [33]:
df_feat.head()

,consumption,hour,dayofweek,month,is_weekend,hour_sin,hour_cos,month_sin,month_cos,lag_1h,lag_24h,lag_168h,rolling_mean_24h,rolling_mean_7d,is_holiday,temperature,humidity
Date,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,51764.567568,0,5,1,1,0.000000,1.000000,0.5,0.866025,NaN,NaN,NaN,NaN,NaN,1,12.4,93.0
2011-01-01 01:00:00,66344.627687,1,5,1,1,0.258819,0.965926,0.5,0.866025,51764.567568,NaN,NaN,NaN,NaN,1,12.3,94.0
2011-01-01 02:00:00,65981.054883,2,5,1,1,0.500000,0.866025,0.5,0.866025,66344.627687,NaN,NaN,NaN,NaN,1,12.2,94.0
2011-01-01 03:00:00,66576.533566,3,5,1,1,0.707107,0.707107,0.5,0.866025,65981.054883,NaN,NaN,NaN,NaN,1,12.2,94.0
2011-01-01 04:00:00,64963.552675,4,5,1,1,0.866025,0.500000,0.5,0.866025,66576.533566,NaN,NaN,NaN,NaN,1,12.2,94.0


In [34]:
df_feat.isna().sum()

consumption           0
hour                  0
dayofweek             0
month                 0
is_weekend            0
hour_sin              0
hour_cos              0
month_sin             0
month_cos             0
lag_1h                1
lag_24h              24
lag_168h            168
rolling_mean_24h     23
rolling_mean_7d     167
is_holiday            0
temperature           1
humidity              1
dtype: int64

In [35]:
df_model = df_feat.dropna()
df_model.head(100)

,consumption,hour,dayofweek,month,is_weekend,hour_sin,hour_cos,month_sin,month_cos,lag_1h,lag_24h,lag_168h,rolling_mean_24h,rolling_mean_7d,is_holiday,temperature,humidity
Date,,,,,,,,,,,,,,,,,
2011-01-08 00:00:00,86403.301624,0,5,1,1,0.000000,1.000000,0.5,0.866025,111609.594412,85716.807571,51764.567568,120015.695552,112329.611000,0,14.0,87.0
2011-01-08 01:00:00,72534.984171,1,5,1,1,0.258819,0.965926,0.5,0.866025,86403.301624,73297.911807,66344.627687,119983.906900,112366.458360,0,14.3,85.0
2011-01-08 02:00:00,69325.030607,2,5,1,1,0.500000,0.866025,0.5,0.866025,72534.984171,70543.526944,65981.054883,119933.136219,112386.362977,0,14.5,84.0
2011-01-08 03:00:00,68554.184257,3,5,1,1,0.707107,0.707107,0.5,0.866025,69325.030607,70242.037536,66576.533566,119862.809000,112398.134708,0,14.5,84.0
2011-01-08 04:00:00,69033.522311,4,5,1,1,0.866025,0.500000,0.5,0.866025,68554.184257,68884.017033,64963.552675,119869.038386,112422.360717,0,14.6,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-11 23:00:00,104858.961164,23,1,1,0,-0.258819,0.965926,0.5,0.866025,126986.088035,105988.140315,107803.446657,119053.812682,118850.778614,0,11.6,93.0
2011-01-12 00:00:00,80345.248738,0,2,1,0,0.000000,1.000000,0.5,0.866025,104858.961164,83494.358024,84784.615561,118922.599796,118824.353812,0,11.2,95.0
2011-01-12 01:00:00,70147.959742,1,2,1,0,0.258819,0.965926,0.5,0.866025,80345.248738,73201.564607,72672.448650,118795.366260,118809.327092,0,10.8,96.0


In [37]:
df_model.to_csv(
    "../dataa/electricity_features_hourly.csv",
    index=True
)